In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#read data set 1 and name it df
df = pd.read_csv ('../input/mobility-report/Global_Mobility_Report -filtered.csv')
#print(df)
#read data set 2 and name it df2
df2 = pd.read_csv ('../input/owid-covid/owid-covid-data2.csv')
#print(df2)
#merge df and df2 usiong method "outer" so that no data is ommitted from any of the 2 data frames
df3= pd.merge(left=df, right=df2, on="date")

#df3.a.fillna(value=0, inplace=True) # This fills all the null values in the columns with 0

#print(df3)

#Export the merged data frame to a csv file

df3.to_csv('mergedDataFrame.csv',index=False)    

#load specifc columns of the merged dataFrame by name
df6 = pd.read_csv(
    './mergedDataFrame.csv',
    header = 0,
    sep = ',',
    usecols = lambda x:x.lower() in ["retail_and_recreation_percent_change_from_baseline", "grocery_and_pharmacy_percent_change_from_baseline", "parks_percent_change_from_baseline", "transit_stations_percent_change_from_baseline", "workplaces_percent_change_from_baseline", "date", "stringency_index", "new_cases"]
)
df6.to_csv('workingCSV.csv', index=False)
print(df6)

In [ ]:
#### !pip install darts

%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


# Read data:
series = TimeSeries.from_csv('./workingCSV.csv',time_col='date')
series.plot()
# Create training and validation sets:
train, val = series.split_after(0.8)
# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)
# create month and year covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(), freq=series.freq_str, periods=1000),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)
cov_train, cov_val = covariates.split_after(pd.Timestamp("19590101"))
my_model = RNNModel(
    model="LSTM",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=300,
    optimizer_kwargs={"lr": 1e-3},
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=14,
    force_reset=True,
    save_checkpoints=True,
)
my_model.fit(
    train_transformed,
    future_covariates=covariates,
    val_series=val_transformed,
    val_future_covariates=covariates,
    verbose=True,
)
def eval_model(model):
    pred_series = model.predict(n=26, future_covariates=covariates)
    plt.figure(figsize=(8, 5))
    series_transformed.plot(label="actual")
    pred_series.plot(label="forecast")
    plt.title("MAPE: {:.2f}%".format(mape(pred_series, val_transformed)))
    plt.legend()


eval_model(my_model)

best_model = RNNModel.load_from_checkpoint(model_name="Air_RNN", best=True)
eval_model(best_model)

#Backesting the RNN
backtest_series = my_model.historical_forecasts(
    series_transformed,
    future_covariates=covariates,
    start=pd.Timestamp("19590101"),
    forecast_horizon=6,
    retrain=False,
    verbose=True,
    
#Plotting the figure
plt.figure(figsize=(8, 5))
#series_transformed.plot(label="actual")
backtest_series.plot(label="backtest")
plt.legend()
plt.title("Backtest, starting Jan 1959, 6-months horizon")
print(
    "MAPE: {:.2f}%".format(
        mape(
            transformer.inverse_transform(series_transformed),
            transformer.inverse_transform(backtest_series),
        )
    )
)
    

In [ ]:
### !pip install darts

%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from matplotlib import pyplot
from pandas import read_csv

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


# Read data:
# Load birth data using read_csv
series = read_csv('../input/workingcsv/workingCSV.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
print(type(series))
#print(series.head())

series = read_csv('../input/workingcsv/workingCSV.csv', header=0, index_col=0)
pyplot.plot(series["retail_and_recreation_percent_change_from_baseline"])
pyplot.show()

# Create training and validation sets:
#training, validation = series.split_after(0.8)
#model=ExplonetiaSmoothing()
#model.fit(training)
#pred=model.predict(len(validation))


In [ ]:
### !pip install darts

%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from matplotlib import pyplot
from pandas import read_csv

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


# Read data:
# Load birth data using read_csv
series = read_csv('../input/workingcsv/workingCSV.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
print(type(series))
#print(series.head())

series = read_csv('../input/workingcsv/workingCSV.csv', header=0, index_col=0)
pyplot.plot(series["grocery_and_pharmacy_percent_change_from_baseline"])
pyplot.show()

# Create training and validation sets:
#training, validation = series.split_after(0.8)
#model=ExplonetiaSmoothing()
#model.fit(training)
#pred=model.predict(len(validation))


In [ ]:
### !pip install darts

%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from matplotlib import pyplot
from pandas import read_csv

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


# Read data:
# Load birth data using read_csv
series = read_csv('../input/workingcsv/workingCSV.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
print(type(series))
#print(series.head())

series = read_csv('../input/workingcsv/workingCSV.csv', header=0, index_col=0)
pyplot.plot(series["parks_percent_change_from_baseline"])
pyplot.show()

# Create training and validation sets:
#training, validation = series.split_after(0.8)
#model=ExplonetiaSmoothing()
#model.fit(training)
#pred=model.predict(len(validation))


In [ ]:
### !pip install darts

%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from matplotlib import pyplot
from pandas import read_csv

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


# Read data:
# Load birth data using read_csv
series = read_csv('../input/workingcsv/workingCSV.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
print(type(series))
#print(series.head())

series = read_csv('../input/workingcsv/workingCSV.csv', header=0, index_col=0)
pyplot.plot(series["transit_stations_percent_change_from_baseline"])
pyplot.show()

# Create training and validation sets:
#training, validation = series.split_after(0.8)
#model=ExplonetiaSmoothing()
#model.fit(training)
#pred=model.predict(len(validation))


In [ ]:
### !pip install darts

%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from matplotlib import pyplot
from pandas import read_csv

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


# Read data:
# Load birth data using read_csv
series = read_csv('../input/workingcsv/workingCSV.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
print(type(series))
#print(series.head())

series = read_csv('../input/workingcsv/workingCSV.csv', header=0, index_col=0)
pyplot.plot(series["workplaces_percent_change_from_baseline"])
pyplot.show()

# Create training and validation sets:
#training, validation = series.split_after(0.8)
#model=ExplonetiaSmoothing()
#model.fit(training)
#pred=model.predict(len(validation))


In [ ]:
### !pip install darts

%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from matplotlib import pyplot
from pandas import read_csv

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


# Read data:
# Load birth data using read_csv
series = read_csv('../input/workingcsv/workingCSV.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
print(type(series))
#print(series.head())

series = read_csv('../input/workingcsv/workingCSV.csv', header=0, index_col=0)
pyplot.plot(series["new_cases"])
pyplot.show()

# Create training and validation sets:
#training, validation = series.split_after(0.8)
#model=ExplonetiaSmoothing()
#model.fit(training)
#pred=model.predict(len(validation))


In [ ]:
### !pip install darts

%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from matplotlib import pyplot
from pandas import read_csv

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


# Read data:
# Load birth data using read_csv
series = read_csv('../input/workingcsv/workingCSV.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
print(type(series))
#print(series.head())

series = read_csv('../input/workingcsv/workingCSV.csv', header=0, index_col=0)
pyplot.plot(series["stringency_index"])
pyplot.show()

# Create training and validation sets:
#training, validation = series.split_after(0.8)
#model=ExplonetiaSmoothing()
#model.fit(training)
#pred=model.predict(len(validation))
